**处理缺失值的方法：** 一般我们采用该列的均值、中位数或众数来填充缺失值。

```python
import pandas as pd
df = pd.read_csv('train.csv')
df.fillna(df.mode().iloc[0], inplace=True)
```

**处理可能指数级增长的数据的方法：** 比如人口、金钱等，可以用对数log处理，来降低数据的波动

```python
df['LogFare'] = np.log1p(df['Fare'])
df['LogFare'].hist() # 直方图
```

**对非数字列进行编码** 比如性别、类别等，需要将其转换为数字，才能用于模型训练。

```python
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)
```

做完数据的清洗之后，就要找出其中的因变量和自变量，将它们各自转换为张量，以便后续的模型训练。

张量最重要的特性就是它的形状（shape），它可以帮助我们理解张量中数据的组织方式。

len(t_indep.shape)，形状的长度称为张量的秩（rank），指的是张量的维度数量。
- 标量的秩为0
- 向量的秩为1
- 矩阵的秩为2
- 依此类推

我们会用随机系数作为初始化参数，给每一个自变量乘以系数，将每行的结果累加起来，就得到了模型的预测值。这个过程中，我们可能会发现某些列的值比较大或比较小，为了处理这种情况，可以用标准化（standardization）或归一化（normalization）等方法，将所有值都缩放到一个相似的范围。



# 一个简化的线性预测模型

1. 找到系统中的一个因变量和一些自变量，比如因变量Y和自变量X1, X2, X3
2. 假定因变量和自变量的线性关系是：Y = w1 * X1 + w2 * X2 + w3 * X3 + b
3. 用随机数初始化系数w1, w2, w3和偏置b
4. 对用于训练的样本数据：
    4.1. 对每个样本，代入自变量X1, X2, X3，代入系数w1, w2, w3和偏置b，计算预测值Y_pred
    4.2. 计算预测值Y_pred与真实值Y之间的差异（loss）
    4.3. 用自动求梯度的方法，计算loss对w1, w2, w3和b的梯度
    4.4. 根据计算得到的梯度，用梯度下降法更新系数w1, w2, w3和偏置b
    4.5. 重复步骤4.1-4.4，直到loss足够小或者达到预设的迭代次数
5. 此时我们得到的系数w1, w2, w3和偏置b，就是我们模型的参数，用它们来预测新的样本数据。


# 随机森林

寻找最佳二元分割：找到一个特征，对数据进行二分法分组，使得分组内的自变量和因变量尽可能趋同，分组外的自变量和因变量也尽可能趋同。
```
# 计算分组内（或外）的相似度
def _side_score(side, y):
    tot = side.sum()
    if tot<=1: return 0
    return y[side].std()*tot
# 计算分组方法的相似度
def score(col, y, split):
    lhs = col<=split
    return (_side_score(lhs,y) + _side_score(~lhs,y))/len(y)
```